In [ ]:
import pandas as pd

import torch

In [ ]:
from torch import nn
from torchvision import models

model = models.vgg16(weights='VGG16_Weights.DEFAULT')
fc = nn.Sequential(
    nn.Linear(7*7*512, 256),
    nn.ReLU(),
    nn.Linear(256,64),
    nn.ReLU(),
    nn.Linear(64,2),
)
model.classifier = fc

In [ ]:
model.load_state_dict(torch.load("/home/snubh104/dev/JSH/vgg16_binary_classifier.pth"))

<All keys matched successfully>

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

pd.set_option('display.max_columns', None)

pids = np.load('/home/snubh104/dev/JSH/npy/piddata.npy')
file_path = "/data/train_data/patients_preprocessed/"
first_only = pd.DataFrame()
for f_name in tqdm(os.listdir(file_path), desc="Get All Subjects First Data"):
    data = pd.read_csv(file_path + f_name)
    if len(data["결과(입실중 사망 여부)"].value_counts()) > 1:
        print(f_name)
    first_only = pd.concat([first_only, data.iloc[0, :]], axis=1, ignore_index=True)
first_only = first_only.transpose()

unique_users = first_only.sort_values(by="입원일").drop_duplicates(subset="환자번호", keep="first")
bmi = pd.read_csv("/home/snubh104/dev/JSH/csv/mv_add_bmi_train.csv",)
unique_bmi = bmi.sort_values(by="입원일").drop_duplicates(subset="환자번호", keep="first")
unique_users = unique_users.sort_values(by="환자번호")
unique_bmi = unique_bmi.sort_values(by="환자번호")
for (index1, row1), (index2, row2) in zip(unique_users.iterrows(), unique_bmi.iterrows()):
    if row1["환자번호"] == float(row2["환자번호"]):
        unique_users.loc[index1, "bmi"] = row2["BMI"]
    else:
        print(index1, index2)
first_only = unique_users

interval = []
for index, row in first_only.iterrows():
    e_ = row["퇴원일"]
    s_ = row["입원일"]
    interval += [(datetime.datetime.strptime(e_, '%Y-%m-%d') - datetime.datetime.strptime(s_, '%Y-%m-%d')).days]

first_only["Interval"] = interval

total = len(first_only)
for label in first_only["결과(입실중 사망 여부)"].value_counts():
    print(label/total)

Get All Subjects First Data:   0%|          | 0/1352 [00:00<?, ?it/s]

0.8709433962264151
0.1290566037735849


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

imbalanced = compute_class_weight(class_weight="balanced",
                                  classes=np.unique(first_only["결과(입실중 사망 여부)"]),
                                  y=first_only["결과(입실중 사망 여부)"])
imbalanced = {i:v for i,v in enumerate(imbalanced)}
imbalanced

{0: 0.5740901213171578, 1: 3.874269005847953}

In [ ]:
import os
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np

target_size= (224, 224)
file_path = "/data/train_data/xray/"
x_ray_list = os.listdir(file_path)

if "Image" in first_only.columns:
    first_only.drop(columns=["Image"], inplace=True)
first_only["Image"] = None
images = []
for index, row in tqdm(first_only.iterrows(), total=len(first_only)):
    p_d_name = f'{str(int(row["환자번호"])).zfill(6)}_{row["입원일"]}'
    if p_d_name in x_ray_list:
        f_name = os.listdir(file_path+p_d_name)[0]
        img = Image.open(f"{file_path+p_d_name}/{f_name}")
        resized = np.array(img.resize(target_size))
        first_only.loc[index, "Image"] = resized.tobytes()
        images += [resized]

to_csv = first_only.dropna()
y = to_csv["결과(입실중 사망 여부)"]
images = np.asanyarray(images)
print(images.shape)
np.save("image.npy", images)
to_csv[features+["Image", "결과(입실중 사망 여부)"]].to_csv("ensemble_data.csv", index=False)
train, test, _, _ = train_test_split(to_csv, y, test_size=0.2, random_state=42, stratify=y)
# train.to_csv("/home/snubh104/dev/JKH/train.csv", index=False)
# test.to_csv("/home/snubh104/dev/JKH/test.csv", index=False)
display(train["결과(입실중 사망 여부)"].value_counts())
display(test["결과(입실중 사망 여부)"].value_counts())

100%|███████████████████████████████████████████████| 1325/1325 [01:00<00:00, 21.99it/s]


(816, 224, 224)


결과(입실중 사망 여부)
0    546
1    106
Name: count, dtype: int64

결과(입실중 사망 여부)
0    137
1     27
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN, KMeansSMOTE, BorderlineSMOTE, SVMSMOTE

features = [
    'aphce_score_점수', '키', '몸무게', '인공호흡기유무', '기도삽관유무', '기관절개술유무', 'E(eye)', 'bmi',
    'M(Motor)', 'V(verbal)', 'VE', 'VT', 'sedation', 'rass_입력값', 'CRP',
    'ABGA', 'PH', 'CBC', 'L2001', 'L2002', 'L2004', 'L2005', 'L20051',
    'L2006', 'L2009', 'L2021', 'L2022', 'L2023', 'L2024', 'L2025', 'L3092',
    'L5112', 'L81101', 'L811011', 'L81104', 'L8186', 'L84301', 'L84304',
    'sbp_입력값', 'dbp_입력값', 'mbp_입력값', 'pr_입력값', 'rr_입력값', 'spo2_입력값',
    'PC(AC)', 'PSV',
    'S/T(spontaneous time)', 'SIMV(PC)', 'SIMV(VC)', 'VC(AC)', 'ppeak_입력값',
    'insp_pr_입력값', 'ps_assip_입력값', 'peep_입력값', 'fio2_입력값', 'frep_set_입력값',
    'tidal_vol_입력값', 'minute_vol_입력값', 'ie_ratio_입력값', 'nasal_prong',
    'facial_mask', 't_cap', 'jet_neb', 'room_air',]

first_only.loc[first_only["mbp_입력값"]==-1, "mbp_입력값"] = first_only["mbp_입력값"].median()
first_only.loc[first_only["pr_입력값"]==-1, "pr_입력값"] = first_only["pr_입력값"].median()
first_only.loc[first_only["rr_입력값"]==-1, "rr_입력값"] = first_only["rr_입력값"].median()
first_only.loc[first_only["spo2_입력값"]==-1, "spo2_입력값"] = first_only["spo2_입력값"].median()

train, test = train_test_split(first_only, test_size=0.1, stratify=first_only["결과(입실중 사망 여부)"])
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')
# train.to_csv("/home/snubh104/dev/JKH/train.csv", index=False)
# test.to_csv("/home/snubh104/dev/JKH/test.csv", index=False)
# oversample = RandomOverSampler(sampling_strategy='minority')
oversample = ADASYN(sampling_strategy=0.75, n_neighbors=35)
train, train_dod = oversample.fit_resample(
    X=train.drop(columns=["결과(입실중 사망 여부)", "성별", "입원일", "퇴원일", "Image"]),
    y=train["결과(입실중 사망 여부)"])
train["결과(입실중 사망 여부)"] = train_dod
train["성별"] = 99
train["입원일"] = ""
train["퇴원일"] = ""

prot_val = ["나이(입실당일)", "성별"]
_X = train.drop(columns=["환자번호", "입원일", "퇴원일", "결과(입실중 사망 여부)", "Interval"]+prot_val)[features]

columns = _X.columns

X = _X.values.astype(float)
y = train["결과(입실중 사망 여부)"].values.reshape(-1,1).astype(int)
y_reg = train["Interval"].values.reshape(-1,1).astype(int)

male_index = test[test["성별"]==0].index
female_index = test[test["성별"]==1].index

X_test = test.drop(columns=["환자번호", "입원일", "퇴원일", "결과(입실중 사망 여부)", "Interval"]+prot_val)[features].values.astype(float)
y_test = test["결과(입실중 사망 여부)"].values.reshape(-1,1).astype(int)
y_reg_test = test["Interval"].values.reshape(-1,1).astype(int)


In [ ]:
from sklearn.model_selection import train_test_split

to_csv = first_only.dropna()
y=to_csv["결과(입실중 사망 여부)"]
train, test, _, _ = train_test_split(to_csv, y, test_size=0.2, random_state=42, stratify=y)
# train.to_csv("/home/snubh104/dev/JKH/train.csv", index=False)
# test.to_csv("/home/snubh104/dev/JKH/test.csv", index=False)
display(train["결과(입실중 사망 여부)"].value_counts())
display(test["결과(입실중 사망 여부)"].value_counts())

결과(입실중 사망 여부)
0    546
1    106
Name: count, dtype: int64

결과(입실중 사망 여부)
0    137
1     27
Name: count, dtype: int64

In [ ]:
clf.predict_proba(to_csv[features].iloc[0].values.reshape(1, -1))

array([[1., 0.]])

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import plotly.express as px

def scoring(X_val, y_val, y_reg_val, clf=None, reg=None, title="Test Dataset score"):
      y_pred = clf.predict(X_val,)
      y_reg_pred = reg.predict(X_val)
      print(title)
      print(classification_report(y_val, y_pred, labels=[0,1]))
      print("AUC ROC: ",roc_auc_score(y_val, y_pred))
      print("\nRMSE:", mean_squared_error(y_reg_val, y_reg_pred, squared=False),
            "\nMAE: ", mean_absolute_error(y_reg_val, y_reg_pred,),
            "\nR2: ", r2_score(y_reg_val, y_reg_pred,))
      y_pred = clf.predict_proba(X_val,)[:,1]
      fpr, tpr, thresholds = roc_curve(
            y_val,
            y_pred,)
      fig = px.area(
            x=fpr, y=tpr,
            title=f'ROC Curve (AUC={roc_auc_score(y_val, y_pred):.4f})',
            labels=dict(x='False Positive Rate', y='True Positive Rate'),
            width=700, height=500
      )
      fig.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
      )

      fig.update_yaxes(scaleanchor="x", scaleratio=1)
      fig.update_xaxes(constrain='domain')
      fig.show()


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

scaler = MinMaxScaler() # StandardScaler() #
X_ = scaler.fit_transform(X, y)
# pca = PCA(n_components=2)
# X_ = pca.fit_transform(X_)

skf = StratifiedKFold(n_splits=3, shuffle=True,)
for train_idx, val_idx in skf.split(X, y):
    clf = ExtraTreesClassifier(n_estimators=750, class_weight=imbalanced, criterion="entropy")
    reg= ExtraTreesRegressor(n_estimators=750,)
    X_train, X_reg_train, y_train, y_reg_train = X[train_idx], X[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X[val_idx], X[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC", roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1]))
    print("RMSE:", mean_squared_error(y_reg_val, y_reg_pred, squared=False),
          "MAE:", mean_absolute_error(y_reg_val, y_reg_pred,),
          "R2:", r2_score(y_reg_val, y_reg_pred,))
    break


/opt/conda/envs/pytorch/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

       alive       0.94      0.95      0.94       346
        dead       0.93      0.91      0.92       257

    accuracy                           0.93       603
   macro avg       0.93      0.93      0.93       603
weighted avg       0.93      0.93      0.93       603

ROC 0.9783180765164977
RMSE: 44.255532532860684 MAE: 13.023471531232726 R2: 0.18224417493486866


In [ ]:
ensemble_data = pd.read_csv("ensemble_data.csv")
images = np.load("image.npy")

In [ ]:
train, test, train_images, test_images = train_test_split(ensemble_data, images, test_size=0.2, random_state=42, stratify=ensemble_data["결과(입실중 사망 여부)"])

In [ ]:
train.shape

(652, 66)

In [ ]:
%%writefile  /home/snubh104/dev/JKH/ensemble.py
import skorch
import torch
from skorch import NeuralNetBinaryClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score
import pandas as pd
import numpy as np
from torch import nn
from torchvision import models
import sys
sys.setrecursionlimit(10**7)
sys.set_int_max_str_digits(0)

ensemble_data = pd.read_csv("ensemble_data.csv")
images = np.load("image.npy")
train, test, train_images, test_images = train_test_split(ensemble_data, images, test_size=0.2, random_state=42, stratify=ensemble_data["결과(입실중 사망 여부)"])

X = train.drop(columns=["결과(입실중 사망 여부)"]).values
X_img = train_images
y = train["결과(입실중 사망 여부)"].values.reshape(-1,1).astype(int)

X_test = test.drop(columns=["결과(입실중 사망 여부)"]).values
X_test_img = test_images
y_test = test["결과(입실중 사망 여부)"].values.reshape(-1,1).astype(int)

class TabularSelector(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X[0]
        return X[:, :-1]

class TabularModel(BaseEstimator):
    def __init__(self):
        super(TabularModel, self).__init__()
        self._estimator_type = "classifier"
        self.model = ExtraTreesClassifier(
            n_estimators=750,
            class_weight={0: 0.574090, 1: 3.874269},
            criterion="entropy")

    def predict(self, X, y=None):
        X = X[:, :65]
        return self.model.predict(X)

    def predict_proba(self, X, y=None):
        X = X[:, :65]
        return self.model.predict_proba(X)

    def fit(self, X, y=None):
        X = X[:, :65]
        return self.model.fit(X, y)


class ImageSelector(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data = X[-1]
        data = data.reshape(len(X[-1]), -1)
        return data

class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()
        self.model = models.vgg16(weights='VGG16_Weights.DEFAULT')
        self.model.classifier = nn.Sequential(
            nn.Linear(7*7*512, 256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,2),)
        self.model.load_state_dict(torch.load("/home/snubh104/dev/JSH/vgg16_binary_classifier.pth"))

    def forward(self, X):
        logit = self.model(X)
        output = torch.argmax(logit, dim=1).float()
        return output.requires_grad_()


class ImageModel(BaseEstimator):
    def __init__(self,):
        super(ImageModel, self).__init__()
        self._estimator_type = "classifier"
        model = VisionModel()
        self.model = NeuralNetBinaryClassifier(
            module=model,
            max_epochs=4,
            optimizer=torch.optim.Adam)

    def transform(self, X, y=None):
        batch_size = len(X)
        # img = X.reshape(batch_size, 224, 224)
        img = torch.from_numpy(X.astype(np.float32)).unsqueeze(1)
        return img.expand(batch_size, 3, 224, 224)

    def predict(self, X, y=None):
        X = self.transform(X[1])
        # self.model.eval()
        return self.model.predict(X)

    def predict_proba(self, X, y=None):
        X = self.transform(X[1])
        # self.model.eval()
        return self.model.predict_proba(X)

    def fit(self, X, y=None):
        img = X[1]
        X = self.transform(img)
        y = torch.from_numpy(y.astype(np.float32))
        # self.model.train()
        return self.model.fit(X, y)

if __name__ == "__main__":
    combined_features = FeatureUnion([
        ('to_tree', TabularSelector()),
        ('to_vision', ImageSelector())
    ])

    tree_pipe = Pipeline([
        ("Feature Selector", combined_features),
        ("Tree", TabularModel())])
    x_ray_pipe = Pipeline([
        # ("Feature Selector", combined_features),
        ("X-ray", ImageModel()),])

    skf = StratifiedKFold(n_splits=3, shuffle=True,)
    for train_idx, val_idx in skf.split(X, y):
        clf = VotingClassifier(
            estimators=[
                ('Tree', tree_pipe),
                ('X-Ray', x_ray_pipe)],
            voting='soft')
        X_train, X_train_img, y_train = X[train_idx], X_img[train_idx], y[train_idx]
        X_val, X_val_img, y_val = X[val_idx], X_img[val_idx], y[val_idx]
        clf.fit((X_train, X_train_img), y_train)
        y_pred = clf.predict((X_val, X_val_img),)
        print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
        print("ROC", roc_auc_score(y_val, clf.predict_proba((X_val, X_val_img),)[:, 1]))


Overwriting /home/snubh104/dev/JKH/ensemble.py


In [ ]:
import os
from PIL import Image
from PIL import UnidentifiedImageError
from tqdm import tqdm
from datetime import datetime as dt

xdata = []
ydata = []
pidlist = []
patient_xray = []
target_size= (224,224)
image_root = "/data/train_data/xray"
inpatient_his = first_only["환자번호"].astype("int").astype("str").str.zfill(6)
for root, dirs, files in tqdm(os.walk(image_root), total=len(dirs)):
    for filename in files:
        if filename.endswith(".png"):
            image_path = os.path.join(root, filename)
            f1, f2, f3, f4, patient_f5, patient_f6 = image_path.split(os.path.sep)
            p_id, p_date = patient_f5.split("_")
            that = dt.now()
            matching_row = first_only[(inpatient_his == p_id)]
            print("1",dt.now()-that)
            if not matching_row.empty:
                death_status = matching_row.iloc[0]['결과(입실중 사망 여부)']
                patient_id = matching_row.iloc[0]['환자번호']

                try:
                    that = dt.now()
                    img = Image.open(image_path)
                    print("2",dt.now() - that)
                    that =  dt.now()
                    resized_img = img.resize(target_size)
                    print("3",dt.now() - that)
                    that = dt.now()
                    xdata.append(np.array(resized_img))
                    ydata.append(death_status)
                    print("4",dt.now() - that)
                    # print(matching_row, death_status)
                except UnidentifiedImageError:
                    print("이미지 인식불가")
                except Exception as e:
                    print(f"오류발생:{e}")

            break

print(xdata)
